In [27]:
## Run ML methods on PanPred and panta outputs 
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import datasets
from sklearn import svm
import random
import os
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from numpy import genfromtxt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
import pandas as pd
import numpy as np
# http://scikit.ml/index.html
from skmultilearn.problem_transform import ClassifierChain

In [28]:
version = '_classifierchain_v1'  # Run feature selection

In [62]:
# # from sklearn.datasets import make_multilabel_classification
# # from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# # from sklearn.multioutput import ClassifierChain
# X, Y = make_multilabel_classification( n_samples=12, n_classes=3, random_state=0)
# X_train, X_test, Y_train, Y_test = train_test_split( X, Y, random_state=0)
# # base_lr = lgb.LGBMClassifier()
# # chain = ClassifierChain(base_lr, order='random', random_state=0)
# # chain.fit(X_train, Y_train).predict(X_test)
# # chain.predict_proba(X_test)
# classifier = ClassifierChain(classifier = lgb.LGBMClassifier())
# classifier.fit(X_train, Y_train)
# predictions = classifier.predict(X_test) 

In [72]:
def run_ML(X, y, data_set, approach, label_column):
    base_dir = '/data/hoan/amromics/prediction/output/predPantaPanPred'+version
    if not os.path.isdir(base_dir):
        os.system('mkdir '+ base_dir)
    score = []
    methods = []
    n_loops = 2
    n_folds = 5
    n_samples = y.shape[0]
    
    for i in range(n_loops):
        cv = KFold(n_splits=n_folds, shuffle=True, random_state = i)
        for fold, (train_idx, test_idx) in enumerate(cv.split(X)):
            print('Run: ', i, ', fold: ', fold)
            X_train = X[train_idx]
            X_test = X[test_idx]
            y_train = y[train_idx]
            y_test = y[test_idx]
            
            # LightGBM
            methods.append('LightGBM')
            print(methods[-1], end =', ')
            classifier = ClassifierChain(classifier = lgb.LGBMClassifier())
            classifier.fit(X_train, y_train)
            predictions = classifier.predict(X_test)
            y_predict = predictions.todense()
            for idx in range(len(label_column)):
                path_dir = base_dir +'/' + data_set+'_'+label_column[idx]+'_run_'+str(i)+'_fold_'+str(fold)+'_'+approach
                np.savetxt(path_dir +"_LightGBM_labels.csv", y_predict[:,idx], delimiter=",")
            # score.append(f1_score(y_predict, y_test, average='macro'))
        
    # # Print statistics
    # n_methods = len(set(methods))
    # score_np = np.array(score)
    # # Each column is a method
    # print(methods[:n_methods])
    # average_score = np.mean(score_np.reshape((n_loops*n_folds, n_methods)), axis=0)
    # print(np.round(average_score, 2))

### Run PanPred 

In [31]:
# pandata = pd.read_csv("PanPred/test_data/gene_presence_absence.csv")

In [32]:
metadata = pd.read_csv('PanPred/test_data/Metadata.csv')
metadata = metadata.set_index(metadata['Isolate'])

In [33]:
accessorygene =  pd.read_csv('PanPred/test_data/AccessoryGene.csv', index_col=0)

In [34]:
populationstructure =  pd.read_csv('PanPred/test_data/PopulationStructure.csv_labelencoded.csv', index_col=0)

In [35]:
new_accessorygene = accessorygene.loc[metadata['Isolate']]

#### Run ML models

In [36]:
# for idx in range(2, 14):
#     y_class = metadata.iloc[:,idx].values
#     print(metadata.columns[idx])
#     y = np.array([1 if y_class[i]=='R' else 0 for i in range(1936)])
#     run_ML(new_accessorygene.values, y, 'Ecoli1936','classic')

In [37]:
# new_accessorygene.head(2)

### Run Panta

In [38]:
sample_isolate = pd.read_csv('/data/hoan/amromics/prediction/data/Ecoli1936metafiles/sample_isolate.csv')
sample_isolate.head(2)
sample2isolate = {}
for idx in range(len(sample_isolate.index)):
    sample2isolate[sample_isolate.iloc[idx,0]+'.contig'] = sample_isolate.iloc[idx,1]

In [39]:
# pa_matrix = pd.read_csv('/data/hoan/amromics/prediction/output/pantaEcoli1936/gene_presence_absence.Rtab', sep='\t', index_col=0).T
pa_matrix = pd.read_csv('/data/hoan/amromics/prediction/output/pantaEcoli1936align_v4/gene_presence_absence.Rtab', sep='\t', index_col=0).T

In [40]:
isolate_index = [sample2isolate[sample] for sample in pa_matrix.index]
metadata_panta = metadata.loc[isolate_index]

#### FS for presence-absence matrix

In [41]:
sel = VarianceThreshold(threshold=0)
pa_matrix = sel.fit_transform(pa_matrix)

In [42]:
pa_matrix.shape

(1653, 73473)

In [43]:
pantaout_dir = '/data/hoan/amromics/prediction/output/pantaEcoli1936align_v4/'
# snp_mat = pd.read_csv(pantaout_dir + 'amrlabelencodermat_VarianceThreshold.csv')
# snp_mat = genfromtxt(pantaout_dir + 'amrlabelencodermat_VarianceThreshold.csv', delimiter=',')
# snp_mat = np.load(pantaout_dir + 'amrlabelencodermat_VarianceThreshold.npy')
snp_mat = np.load(pantaout_dir + 'amrlabelencodermat.npy')

In [44]:
# snp_mat.shape
# full_matrix =  np.concatenate((pa_matrix, snp_mat), axis=1)
# scaler = StandardScaler()
# full_matrix = scaler.fit_transform(full_matrix)

In [69]:
Y = []; label_column = [];
for idx in range(2, 14):
    y_class = metadata_panta.iloc[:,idx].values
    label_column.append(metadata_panta.columns[idx])
    y = np.array([1 if y_class[i]=='R' else 0 for i in range(len(y_class))])
    Y.append(y)
    # Run unimodal gene
    # run_ML(pa_matrix, y, 'Ecoli1936'+'_'+metadata_panta.columns[idx],'pantaPangenome', False, 'mutual_info_classif', None)
    # run_ML(full_matrix, y, 'Ecoli1936'+'_'+metadata_panta.columns[idx],'pantaPangenome', False, 'mutual_info_classif', None)
    # run_ML(snp_mat, y, 'Ecoli1936'+'_'+metadata_panta.columns[idx],'pantaSnp', True, 'chi2')
    # run_ML(pa_matrix, y, 'Ecoli1936'+'_'+metadata_panta.columns[idx],'pantaCombine', False, 'mutual_info_classif', snp_mat)
    # run_ML(pa_matrix, y, 'Ecoli1936'+'_'+metadata_panta.columns[idx],'pantaCombineScale', True, 'chi2', snp_mat)
Y = np.array(Y).transpose()

In [74]:
# run_ML(pa_matrix[:100,], Y[:100,], 'Ecoli1936','pantaPanClassifierChain', label_column)
run_ML(pa_matrix, Y, 'Ecoli1936','pantaPanClassifierChain', label_column)

Run:  0 , fold:  0
LightGBM, Run:  0 , fold:  1
LightGBM, Run:  0 , fold:  2
LightGBM, Run:  0 , fold:  3
LightGBM, Run:  0 , fold:  4
LightGBM, Run:  1 , fold:  0
LightGBM, Run:  1 , fold:  1
LightGBM, Run:  1 , fold:  2
LightGBM, Run:  1 , fold:  3
LightGBM, Run:  1 , fold:  4
LightGBM, 

In [ ]:
# ## Run PanPred on panta isolate
# pa_matrixPanPred = accessorygene.loc[isolate_index]
# for idx in range(2, max_idx_amr):
#     y_class = metadata_panta.iloc[:,idx].values
#     print(metadata_panta.columns[idx])
#     y = np.array([1 if y_class[i]=='R' else 0 for i in range(len(y_class))])
#     run_ML(pa_matrixPanPred.values, y, 'Ecoli1936'+'_'+metadata_panta.columns[idx],'PanPred' )

In [ ]:
# ## Run PanPred on panta isolate: + population structure
# pa_matrixPanPred = accessorygene.loc[isolate_index]
# ps_matrixPanPred = populationstructure.loc[isolate_index]
# combinematrixPanPred = np.concatenate((pa_matrixPanPred.values, ps_matrixPanPred.values), axis=1)

In [ ]:
# scaler = StandardScaler()
# scaled_combinematrixPanPred = scaler.fit_transform(combinematrixPanPred)

In [ ]:
# for idx in range(2, max_idx_amr):
#     y_class = metadata_panta.iloc[:,idx].values
#     print(metadata_panta.columns[idx])
#     y = np.array([1 if y_class[i]=='R' else 0 for i in range(len(y_class))])
#     run_ML(scaled_combinematrixPanPred, y, 'Ecoli1936'+'_'+metadata_panta.columns[idx],'PanPredCombine' )